## Things to do with the transformer module
#### 1) Block for the achitecture
####     2) Transformer block finilized with interactable modules

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import os
import torchmetrics
import datetime
from functools import partial
from torch import einsum
from einops import rearrange, repeat
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.models.registry import register_model
from timm.models.vision_transformer import _cfg
from ConvFFN import CONVFFN

c:\Users\madhava\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
DropPath(0.3)

DropPath(drop_prob=0.300)

In [3]:
from math import sqrt
sqrt(1)

1.0

In [4]:
from torch.distributed.algorithms._checkpoint.checkpoint_wrapper import (
 checkpoint_wrapper,
 CheckpointImpl )

import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler

from torch.distributed.fsdp import (
    FullyShardedDataParallel as FSDP,
    MixedPrecision,
    BackwardPrefetch,
    ShardingStrategy,
    FullStateDictConfig,
    StateDictType,
)
from torch.distributed.fsdp.wrap import (
    transformer_auto_wrap_policy,
    enable_wrap,
    wrap,
)
from CNNbackbone import Make_CNN_BACK_BONE

In [5]:
class OverlapPatchEmbedding(nn.Module):

    def __init__(self,image_size=224,patch_size=7,stride=4,in_chans=3,embed_dim=768):
        super().__init__()

        self.image_size=(image_size,image_size)
        self.patch_size=(patch_size,patch_size)

        assert max(self.patch_size)>stride, "larger than stride"

        self.image_size=(image_size,image_size)
        self.patch_size=(patch_size,patch_size)

        self.H,self.W=self.image_size[0]//stride,self.image_size[1]//stride

        self.proj=nn.Conv2d(in_chans,embed_dim,kernel_size=self.patch_size,stride=stride,
                            padding=(self.patch_size[0]//2,self.patch_size[1]//2))
        self.norm=nn.LayerNorm(embed_dim)

        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()
    
    def forward(self, x):
        x = self.proj(x)
        print(x.shape,"Projection")
        _, _, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = self.norm(x)

        return x, H, W



OverlapPatchEmbedding(
                image_size=self.image_size if i==0 else self.image_size//(2**(i + 1)),
                patch_size= self.patch_size if i==0 else 3,
                stride= 4 if i==0 else 2,
                in_chans=3 if i==0 else self.embeded_dimesion[i-1],
                embed_dim=self.embeded_dimesion[i]
            )

In [6]:
class SmallBlock_attention(nn.Module):
    def __init__(self,dim,num_heads=4,qkv_bias=False,qk_scale=None,attn_drop=0.,proj_drop=0.):
        self.dim=dim
        self.num_heads=4
        
        self.q=nn.Linear(dim,dim,bias=qkv_bias)
        self.kv=nn.Linear(dim,dim*2,bias=qkv_bias)
        self.qscale=self.num_heads**-0.5
        self.attn_drop=nn.Dropout(attn_drop)
        self.proj_drop=nn.Dropout(proj_drop)
        self.norm=nn.LayerNorm(dim)
        
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()
    
    def forward(self,k,v):
        stacked=torch.stack([k,v],dim=0) # b d n c -> qk b d n c
        x=rearrange(stacked,'qk b d n c -> b n (d c qk)')

        q= self.q(x)
        
        kv=self.kv(x)

        q=rearrange(q,'b n (d c) -> b d n c',d=self.dim)

        kv=rearrange(kv,'b n (qk d c) -> qk b d n c',qk=2,d=self.num_heads)
        
        #seprate k and v

        k,v=kv[0],kv[1]
        #attention mechanism and send only attention map 

        attn=einsum('b i j l, b i k m -> b i j k',q,k)*self.qscale

        attn=attn.softmax(dim=-1)
        attn=self.attn_drop(attn)
        attn= einsum('b d l n, b d n j-> b d l j',attn,v)
        attn=rearrange(attn,'b d n c -> b n (c d)')

        final=x+attn
        final=self.norm(final)
        final=rearrange(final,'b n (qk d c) -> qk b d n c',qk=2,d=self.num_heads)
        return 

In [7]:
import torch

In [8]:
class Attention(nn.Module):
    def __init__(self, dim,num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., sr_ratio=1, linear=False):
        super().__init__()
        self.num_heads = num_heads
        self.qkv_bias = qkv_bias
        self.dim=dim

        self.head_dim= dim// num_heads
        self.qk_scale = qk_scale or self.head_dim ** -0.5

        self.sr_ratio = sr_ratio

        self.linear = linear

        self.q= nn.Linear(dim, dim, bias=qkv_bias)
        self.kv= nn.Linear(dim, dim*2, bias=qkv_bias)
        self.proj= nn.Linear(dim, dim)  
        self.attn_drop= nn.Dropout(attn_drop)
        self.proj_drop= nn.Dropout(proj_drop)
        self.k_learnable=nn.Parameter(torch.randn(dim), requires_grad=True)
        self.v_learnable=nn.Parameter(torch.randn(dim), requires_grad=True)
        if not linear:
            if self.sr_ratio > 1:
                self.sr = nn.Conv2d(dim, dim, kernel_size=sr_ratio, stride=sr_ratio)
                self.norm = nn.LayerNorm(dim)
        else:
            self.adaptive_pooling=nn.AdaptiveAvgPool2d(7)
            self.norm = nn.LayerNorm(dim)
            self.act=nn.GELU()
            self.sr = nn.Conv2d(dim, dim, kernel_size=1, stride=1)
    
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()
    
    def forward(self,x,H,W):
        B,N,C=x.shape

        qv=self.q(x)
        qv=rearrange(qv,'b n (d c) -> b d n c',d=self.num_heads)

        if not self.linear:
            if self.sr_ratio >1:
                x_=rearrange(x,'B (H W) C -> B C H W',H=H,W=W)
                x_=rearrange(self.sr(x_),'B C H W -> B (H W) C')
                x_=self.norm(x_)
                kv=self.kv(x_)
                kv=rearrange(kv,'B N (d c qk) -> qk B d N c',d=self.num_heads,c=C//self.num_heads)
            else:
                kv=self.kv(x)
                kv=rearrange(kv,'b n (d c qk) -> qk b d n c',d=self.num_heads,c=C//self.num_heads)
            
        else:
            x_=rearrange(x,'b (H W) C -> b C H W', H=H,W=W)
            x_=self.sr(self.adaptive_pooling(x_))
            x_=rearrange(x_,'b c h w -> b (h w) c')
            x_=self.norm(x_)
            x_=self.act(x_)
            kv=rearrange(kv,'b n (d c qk) -> qk b d n c',d=self.num_heads,c=C//self.num_heads)

        k,v=kv[0],kv[1]


        l_k=repeat(self.k_learnable,'h1 -> b h1',b=B) # Batch h1
        

        l_v=repeat(self.v_learnable,'h1  -> b h1 ',b=B) # Batch h1 


        l_v=rearrange(l_v,"b (a d)-> b a d",a=self.num_heads)
        l_k=rearrange(l_k,"b (a d)-> b a d",a=self.num_heads)

        k=rearrange(k,'b head n c -> b n head c')
        v=rearrange(v,'b head n c -> b n head c')

        k=einsum('b n h c, b p q -> b p n q',k,l_k)
        v=einsum('b n h c, b p q -> b p n q',v,l_v)


        attn=einsum('b i j l, b i k m -> b i j k',qv,k)*self.qk_scale

        

        attn=attn.softmax(dim=-1)

        attn=self.attn_drop(attn)


        attn= einsum('b d l n, b d n j-> b d l j',attn,v)
        attn=rearrange(attn,'b d n c -> b n (c d)')
        
        
        x= self.proj(attn)
        x=self.proj_drop(x)

        return x
            


In [9]:
class Block(nn.Module):
    def __init__(self,dim,heads,qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., sr_ratio=1, linear=False,mlp_ratio=4.0,norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim=dim
        self.heads=heads
        self.qkv_bias=qkv_bias
        self.qk_scale=qk_scale
        self.attn_drop=DropPath(attn_drop) if attn_drop>0 else nn.Identity()
        self.proj_drop=DropPath(proj_drop) if proj_drop>0 else nn.Identity()
        self.sr_ratio=sr_ratio
        self.linear=linear
        

        self.norm1=norm_layer(self.dim)

        self.attn=Attention(dim=self.dim,num_heads=self.heads,qkv_bias=self.qkv_bias,qk_scale=self.qk_scale,
                            attn_drop=attn_drop,proj_drop=proj_drop,sr_ratio=self.sr_ratio,linear=self.linear)
        
        self.norm2=norm_layer(self.dim)

        self.mlp_dim= int(self.dim * mlp_ratio)

        self.conv_mlp= CONVFFN(in_channels=self.dim,hidden_dimension=self.mlp_dim)

    
    def forward(self,x,H,W):

        x=x + self.attn_drop(self.attn(self.norm1(x),H,W))
        x=x + self.proj_drop(self.conv_mlp(self.norm2(x),H,W))
        print(x.shape,"Layer1 norm")
        return x

        

In [10]:
class LearnablePatchAttentionModel(nn.Module):

    def __init__(self,image_size=224,patch_size=7,embeded_dimesion=[64,128,320,512],depth=[3,6,40,3],
                 num_heads=[1,2,5,8], sr_ratio=[8,4,2,1],norm_layer=nn.LayerNorm,qkv_bias=True,qk_scale=None,attn_drop=0.1,proj_drop=0.1,
                 stages=4,mlp_ratio=[8.0,8.0,4.0,4.0],linear=True):
        
        super().__init__()

        self.image_size=image_size
        self.patch_size=patch_size
        self.embeded_dimesion=embeded_dimesion
        self.depth=depth
        self.num_heads=num_heads
        self.norm_layer=norm_layer
        self.qkv_bias=qkv_bias
        self.qk_scale=qk_scale
        self.attn_drop=attn_drop
        self.proj_drop=proj_drop

        self.stages=stages
        self.mlp_ratio=mlp_ratio

        dpr=[x.item() for x in torch.linspace(0,self.proj_drop,sum(self.depth))]
        cur=0
        print(dpr)
        
        for i in range(self.stages):

            

            patch_embedding=OverlapPatchEmbedding(
                image_size=self.image_size if i==0 else self.image_size//(2**(i + 1)),
                patch_size= 7 if i==0 else 3,
                stride= 4 if i==0 else 2,
                in_chans=3 if i==0 else self.embeded_dimesion[i-1],
                embed_dim=self.embeded_dimesion[i]
            )

            if i==0:
                #learnable_weights=nn.Parameter(
                 #   torch.randn(2,3196,64),
                  #  requires_grad=True
                #) verision 2
                pass
            attn=nn.ModuleList(
                [
                    Block(
                        dim=embeded_dimesion[i],
                        qkv_bias=True,
                        qk_scale=None,
                        attn_drop=attn_drop,
                        proj_drop=dpr[cur+j],
                        sr_ratio=sr_ratio[i],
                        linear=linear,
                        mlp_ratio=self.mlp_ratio[i],
                        heads=self.num_heads[i]
                        
                    ) for j in range(self.depth[i])
                ]
            )

            norm=norm_layer(embeded_dimesion[i])



            cur+=self.depth[i]

            setattr(self,f'patch_embed{i +1}',patch_embedding)
            """if i==0:
                setattr(self,f'learnable_weights_{i +1}',learnable_weights) version 2"""

            setattr(self,f'block{i +1}',attn)
            setattr(self,f'norm{i +1}',norm)

        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()
    

    def forward_features(self,x):
        B=x.shape[0]
        c=0

        for i in range(self.stages):
            patch_embedding=getattr(self,f'patch_embed{i +1}')
            Block=getattr(self,f'block{i +1}')
            norm=getattr(self,f'norm{i +1}')

            x,H,W=patch_embedding(x)
            for blk in Block:
                x=blk(x,H,W)

            
            x=norm(x)

            if i!=self.stages-1:
                x=x.reshape(B,H,W,-1).permute(0,3,1,2).contiguous()
            

        return x
    
    def forward(self,x):
        return self.forward_features(x)

In [11]:
a=torch.randn(1,3196,64)
lora1=torch.randn(3196,8)
lora2=torch.randn(8,64)

a1=(lora1@lora2)*a
a1.shape

torch.Size([1, 3196, 64])

In [12]:
class LORA_LearnablePatchAttentionModel_classification(nn.Module):
    def __init__(self,model,rank_of_matrix,alpha,num_classes=3):
        super().__init__()
        self.model=model
        for i in self.model.parameters():
            i.requires_grad=False
        
        self.rank_of_matrix=rank_of_matrix
        
        self.num_stages=self.model.stages

        self.alpha=alpha
        self.num_classes=num_classes

        self.stage1_a=nn.Parameter(
            torch.randn(rank_of_matrix,64),
            requires_grad=True
        )
        self.stage1_b=nn.Parameter(
            torch.randn(3136,rank_of_matrix),
            requires_grad=True
        )

        self.stage2_a=nn.Parameter(
            torch.randn(rank_of_matrix,128),requires_grad=True)
        
        self.stage2_b=nn.Parameter(
            torch.randn(784,rank_of_matrix),requires_grad=True
        )

        self.stage3_a=nn.Parameter(
                torch.randn(rank_of_matrix,256),requires_grad=True
        )

        self.stage3_b=nn.Parameter(
                torch.randn(196,rank_of_matrix),requires_grad=True
        )

        self.stage4_a=nn.Parameter(
                torch.randn(rank_of_matrix,512),requires_grad=True
        )
        
        self.stage4_b=nn.Parameter(
                torch.randn(49,rank_of_matrix),requires_grad=True
        )
        
        


        self.mlp_head=nn.Sequential(
            nn.Linear(512,128),
            nn.LayerNorm(128),
            nn.GELU(),

            nn.Linear(128,self.num_classes)
        )

        self._init_weights()
    


    def forward_once(self,x):

        for i in range(self.num_stages):
            patch_embedding=getattr(self.model,f'patch_embed{i +1}')
            block=getattr(self.model,f'block{i +1}')
            norm=getattr(self.model,f'norm{i +1}')
            
            x,H,W=patch_embedding(x)
            x_lora=x

            #forward pass of main model

            for blk in block:
                x=blk(x,H,W)
            x=norm(x)
            print(H,W)
            
            A=getattr(self,f'stage{i+1}_a')
            B=getattr(self,f'stage{i+1}_b')
            print(B.shape,A.shape,x_lora.shape,(B@A).shape)
            x_lora=(B@A)*x_lora

            x+=self.alpha*x_lora
            print(x.shape)
            print("lora added")

            if i!=self.num_stages-1:
                x=rearrange(x, 'b (h w) c-> b c h w', h=H, w=W)
                x=x.contiguous()

            
    def forward(self,x):
        x=self.forward_once(x)
        x=self.mlp_head(x)
        return x

    def _init_weights(self):
        for i in range(self.num_stages):
            A=getattr(self,f'stage{i+1}_a')
            B=getattr(self,f'stage{i+1}_b')
            nn.init.kaiming_normal_(A)
            nn.init.zeros_(B)
        

In [13]:
model=LearnablePatchAttentionModel(
    num_heads=[1,2,5,8],
    embeded_dimesion=[64,128,320,512],
    depth=[3,6,40,3],
    patch_size=7,
    norm_layer=partial(nn.LayerNorm,eps=1e-6),
    mlp_ratio=[8, 8, 4, 4],
    sr_ratio=[8,4,2,1],linear=False
)

[0.0, 0.0019607844296842813, 0.003921568859368563, 0.005882353521883488, 0.007843137718737125, 0.009803921915590763, 0.011764707043766975, 0.013725491240620613, 0.01568627543747425, 0.01764705963432789, 0.019607843831181526, 0.021568628028035164, 0.02352941408753395, 0.02549019828438759, 0.027450982481241226, 0.029411766678094864, 0.0313725508749485, 0.03333333507180214, 0.03529411926865578, 0.037254903465509415, 0.03921568766236305, 0.04117647185921669, 0.04313725605607033, 0.045098040252923965, 0.0470588281750679, 0.04901961237192154, 0.05098038911819458, 0.05294117331504822, 0.054901961237192154, 0.05686274543404579, 0.05882352963089943, 0.06078431382775307, 0.0627450942993164, 0.06470587849617004, 0.06666666269302368, 0.06862744688987732, 0.07058823108673096, 0.0725490152835846, 0.07450979948043823, 0.07647058367729187, 0.0784313753247261, 0.08039215952157974, 0.08235294371843338, 0.08431372791528702, 0.08627451211214066, 0.0882352963089943, 0.09019608050584793, 0.09215686470270157

In [14]:
import torch
import copy
l=torch.load('pvt_v2_b2.pth')
new_state_dict=copy.deepcopy(model.state_dict())

In [15]:
def match_with_keys(value):
    value=value.split('.')
    

In [16]:
for i in model.state_dict().keys():
    if i in l.keys():
        new_state_dict[i]=l[i]

In [18]:
for named,parameters in model.named_parameters():
    print(named in l.keys(),named)

True patch_embed1.proj.weight
True patch_embed1.proj.bias
True patch_embed1.norm.weight
True patch_embed1.norm.bias
True block1.0.norm1.weight
True block1.0.norm1.bias
False block1.0.attn.k_learnable
False block1.0.attn.v_learnable
True block1.0.attn.q.weight
True block1.0.attn.q.bias
True block1.0.attn.kv.weight
True block1.0.attn.kv.bias
True block1.0.attn.proj.weight
True block1.0.attn.proj.bias
True block1.0.attn.sr.weight
True block1.0.attn.sr.bias
True block1.0.attn.norm.weight
True block1.0.attn.norm.bias
True block1.0.norm2.weight
True block1.0.norm2.bias
False block1.0.conv_mlp.dwconv1.weight
False block1.0.conv_mlp.dwconv1.bias
False block1.0.conv_mlp.pwconv1.weight
False block1.0.conv_mlp.pwconv1.bias
False block1.0.conv_mlp.norm1.weight
False block1.0.conv_mlp.norm1.bias
False block1.0.conv_mlp.dwconv2.weight
False block1.0.conv_mlp.dwconv2.bias
False block1.0.conv_mlp.pwconv2.weight
False block1.0.conv_mlp.pwconv2.bias
False block1.0.conv_mlp.norm2.weight
False block1.0.co

In [16]:
torch.save(new_state_dict,"ovtparameters.pth")

In [17]:
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [20]:
for i in l.keys():
    if i in model.state_dict().keys():
        if False in l[i]==model.state_dict()[i]:
            print( False in l[i]==model.state_dict()[i],'\n')